In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, d_model, img_size, patch_size, n_channels) -> None:
        super().__init__()

        self.d_model = d_model  # Dimensionality of Model
        self.img_size = img_size  # Image Size
        self.patch_size = patch_size  # Patch Size
        self.n_channels = n_channels  # Number of Channels

        self.linear_project = nn.Conv2d(
            self.n_channels,
            self.d_model,
            kernel_size=self.patch_size,
            stride=self.patch_size,
        )

    # B: Batch Size
    # C: Image Channels
    # H: Image Height
    # W: Image Width
    # P_col: Patch Column
    # P_row: Patch Row
    def forward(self, x):
        x = self.linear_project(x)  # (B, C, H, W) -> (B, d_model, P_col, P_row)
        x = x.flatten(2)  # (B, d_model, P_col, P_row) -> (B, d_model, P)
        x = x.transpose(1, 2)  # (B, d_model, P) -> (B, P, d_model)
        return x

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length) -> None:
        super().__init__()

        # Classification token.
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model))

        # Creating positional encoding.
        pe = torch.zeros(max_seq_length, d_model)

        for pos in range(max_seq_length):
            for i in range(d_model):
                if i % 2 == 0:
                    pe[pos][i] = np.sin(pos / (10000 ** (i / d_model)))
                else:
                    pe[pos][i] = np.cos(pos / (10000 ** ((i - 1) / d_model)))

        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        # Expand to have class token for every image in batch.
        tokens_batch = self.cls_token.expand(x.size()[0], -1, -1)

        # Adding class tokens to the beginning of each embedding.
        x = torch.cat((tokens_batch, x), dim=1)

        # Add positional encoding to embeddings.
        return x + self.pe

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, d_model, head_size) -> None:
        super().__init__()
        self.head_size = head_size

        self.query = nn.Linear(d_model, head_size)
        self.key = nn.Linear(d_model, head_size)
        self.value = nn.Linear(d_model, head_size)

    def forward(self, x):
        # Obtaining Queries, Keys, and Values.
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

        # Dot Product of Queries and Keys.
        attention = Q @ K.transpose(-2, -1)

        # Scaling.
        attention = attention / (self.head_size**0.5)
        attention = torch.softmax(attention, dim=-1)
        return attention @ V

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads) -> None:
        super().__init__()
        self.head_size = d_model // n_heads
        self.W_o = nn.Linear(d_model, d_model)
        self.heads = nn.ModuleList(
            [AttentionHead(d_model, self.head_size) for _ in range(n_heads)]
        )

    def forward(self, x):
        # Combine attention heads.
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        return self.W_o(out)

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, n_heads, r_mlp=4) -> None:
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads

        # Sub-Layer 1 Normalization.
        self.ln1 = nn.LayerNorm(d_model)

        # Multi-Head Attention.
        self.mha = MultiHeadAttention(d_model, n_heads)

        # Sub-Layer 2 Normalization.
        self.ln2 = nn.LayerNorm(d_model)

        # Multilayer Perception.
        self.mlp = nn.Sequential(
            nn.Linear(d_model, d_model * r_mlp),
            nn.GELU(),
            nn.Linear(d_model * r_mlp, d_model),
        )

    def forward(self, x):
        # Residual Connection After Sub-Layer 1.
        out = x + self.mha(self.ln1(x))

        # Residual Connection After Sub-Layer 2.
        return out + self.mlp(self.ln2(out))

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(
        self, d_model, n_classes, img_size, patch_size, n_channels, n_heads, n_layers
    ) -> None:
        super().__init__()

        assert (
            img_size[0] % patch_size[0] == 0 and img_size[1] % patch_size[1] == 0
        ), "img_size dimensions must be divisible by patch_size dimensions"
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model  # Dimensionality of model.
        self.n_classes = n_classes  # Number of classes.
        self.img_size = img_size  # Image size.
        self.patch_size = patch_size  # Patch size.
        self.n_channels = n_channels  # Number of channels.
        self.n_heads = n_heads  # Number of attention heads.

        self.n_patches = (self.img_size[0] * self.img_size[1]) // (
            self.patch_size[0] * self.patch_size[1]
        )
        self.max_seq_length = self.n_patches + 1

        self.patch_embedding = PatchEmbedding(
            self.d_model, self.img_size, self.patch_size, self.n_channels
        )
        self.positional_encoding = PositionalEncoding(self.d_model, self.max_seq_length)
        self.transformer_encoder = nn.Sequential(
            *[TransformerEncoder(self.d_model, self.n_heads) for _ in range(n_layers)]
        )

        # Classification MLP.
        self.classifier = nn.Sequential(
            nn.Linear(self.d_model, self.n_classes), nn.Softmax(dim=-1)
        )

    def forward(self, images):
        x = self.patch_embedding(images)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = self.classifier(x[:, 0])
        return x

In [16]:
# Load dataset.

fname = "data/LSWMD_dataset.pkl"  # If it doesn't exists run 'LSWMD-explore.ipynb'.
df = pd.read_pickle(fname)

X = np.array([x for x in df["X"].to_numpy()])
Y = df["Y"].to_numpy(dtype=np.uint8)

# Split data into training and testing.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# Convert numpy data to pytorch tensors.
train_tensor_data = torch.from_numpy(X_train).unsqueeze(1).float()  # Add channel of 1.
train_tensor_labels = torch.from_numpy(y_train).long()

test_tensor_data = torch.from_numpy(X_test).unsqueeze(1).float()  # Add channel of 1.
test_tensor_labels = torch.from_numpy(y_test).long()

# Load tensor data.
BATCH_SIZE = 128

train_dataset = TensorDataset(train_tensor_data, train_tensor_labels)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(test_tensor_data, test_tensor_labels)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [22]:
# Training parameters.

d_model = 12
n_classes = len(np.unique(y_train))
img_size = X_train[0].shape
patch_size = (12, 12)
n_channels = 1
n_heads = 4
n_layers = 3
batch_size = BATCH_SIZE
epochs = 10
alpha = 0.01

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transformer = VisionTransformer(
    d_model, n_classes, img_size, patch_size, n_channels, n_heads, n_layers
).to(device)

optimizer = Adam(transformer.parameters(), lr=alpha)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    training_loss = 0.0
    for batch_data, batch_labels in train_loader:
        inputs, labels = batch_data.to(device), batch_labels.to(device)

        optimizer.zero_grad()

        outputs = transformer(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        training_loss += loss.item()

    print(
        f"Epoch {epoch + 1}/{epochs} loss: {training_loss  / len(train_loader) :.3f}"
    )

Using device:  cuda (Tesla V100-PCIE-32GB)
Epoch 1/10 loss: 0.570
Epoch 2/10 loss: 0.570
Epoch 3/10 loss: 0.570
Epoch 4/10 loss: 0.570


KeyboardInterrupt: 

In [15]:
correct = 0
total = 0

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        images, labels = batch_data.to(device), batch_labels.to(device)

        outputs = transformer(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"\nModel Accuracy: {100 * correct // total} %")


Model Accuracy: 93 %
